In [1]:
import requests

post_data = {
    "pageSize": 1100,
    "curPage": 1,
}

r=requests.post("https://beian.cfdi.org.cn/CTMDS/pub/PUB010100.do?method=handle06&_dt=20210510122555",
                          data=post_data)

In [2]:
import json
import pandas as pd
data_json = json.loads(r.text)
df = pd.DataFrame(data_json['data'])
df.to_csv("data.csv", index=False, encoding='utf-8-sig')

In [3]:
def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

list(split(range(11), 2))

[range(0, 6), range(6, 11)]

In [4]:
from lxml import etree
import html

list_companyID = df["companyId"].to_list()

d = {}
for i, company_id in enumerate(list_companyID):
    print(i, company_id)
    url = "https://beian.cfdi.org.cn/CTMDS/pub/PUB010100.do?method=handle07&compId=%s" % company_id
    detail = requests.get(url)
    root = etree.HTML(detail.content.decode('utf-8'))
    # hosp_level_div = root.xpath("/html/body/div[1]/div[1]/div/div[3]/div")
    # hosp_level = ''.join(hosp_level_div[0].itertext()).strip()
    # d[company_id] = hosp_level
    # print(company_id, hosp_level)
    try:
        doctor_table_div =  root.xpath("/html/body/div[1]/div[2]/div")
        elements = doctor_table_div[0].xpath('.//tr/td//text()')
        doctors = list(split(elements, int(len(elements)/3)))
        df_doctors = pd.DataFrame(doctors, columns = ['学科', '姓名', "职称"])
        df_doctors["companyId"] = company_id
        if i == 0:
            df_combined = df_doctors
        else:
            df_combined = pd.concat([df_combined, df_doctors], axis=0)
    except:
        continue


In [5]:
print(df_combined)
df_combined.to_csv('doctors.csv', index=False, encoding='utf-8-sig')

                       学科   姓名        职称                         companyId
0                内科-其他-肝病  丁惠国   教授,主任医师  54533298C0A8023378B9E559FDE7DF5E
1                内科-其他-肝病  陈新月   教授,主任医师  54533298C0A8023378B9E559FDE7DF5E
2                内科-其他-肝病   张晶   主任医师,教授  54533298C0A8023378B9E559FDE7DF5E
3                内科-其他-肝病  孟庆华   教授,主任医师  54533298C0A8023378B9E559FDE7DF5E
4                内科-其他-肝病   徐斌  副教授,主任医师  54533298C0A8023378B9E559FDE7DF5E
..                    ...  ...       ...                               ...
0                肿瘤科-肿瘤内科  刘义冰     副主任医师  46D8EA3AC0A802336AC3DF28D37D87F4
0               内科-呼吸内科专业  王红嫚      主任医师  1F2F64E9C0A802336243E44106F0A5B1
1                内科-内分泌专业  张勇军      主任医师  1F2F64E9C0A802336243E44106F0A5B1
2                     肿瘤科  周立霞     副主任医师  1F2F64E9C0A802336243E44106F0A5B1
0   其它-Ⅰ期临床试验研究室-I期药物临床试验  丁存刚      副研究员  CD5E4AEEC0A802333F33F9319F75D8B2

[26431 rows x 4 columns]


In [39]:
(pd.DataFrame.from_dict(data=d, orient='index')
   .to_csv('dict_hplevel.csv', header=False, encoding='utf-8-sig'))